In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torchvision
import torch
from PIL import Image
import torchvision.transforms as T
import cv2
import matplotlib.pyplot as plt
import os
import pandas as pd
import math

## **bounding box 좌표 찾고 crop해서 내보내는 코드**

In [ ]:
# 미리 학습된 Mask R-CNN 모델 로드
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

# 모델을 평가 모드로 설정
model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|██████████| 170M/170M [00:00<00:00, 259MB/s]


MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [ ]:
# bounding box가 여러 개가 나올 수 있으므로, 확률 값이 가장 큰 bounding box의 index 값을 추출하는 함수 정의
def find_max_index(tensor):
    max_value, max_index = torch.max(tensor, dim=0)
    return max_index.item()

In [ ]:
image = Image.open(path + file_list_py[i])
  # 이미지를 Torch Tensor로 변환
  transform = T.Compose([T.ToTensor()])
  input_image = transform(image)
  # 이미지 배치(batch)로 변환
  input_image = input_image.unsqueeze(0)

  # 모델 예측 수행
  with torch.no_grad():
    prediction = model(input_image)

  boxes = prediction[0]['boxes'] # bounding box 좌표
  scores = prediction[0]['scores'] #객체의 확률 값들

  # 확률 값이 가장 큰 bounding box 좌표 출력
  bounding_box = boxes[find_max_index(scores)]

  # 텐서에서 바운딩 박스 좌표 얻기
  x_min, y_min, x_max, y_max = bounding_box

  # 텐서를 정수로 반환(인덱싱 하려고)
  x_min = math.floor(x_min.item())
  y_min = math.floor(y_min.item())
  x_max = math.floor(x_max.item())
  y_max = math.floor(y_max.item())

  # 이미지 읽기
  image = cv2.imread(path + file_list_py[i])

  # 바운딩 박스 좌표를 이용하여 이미지 crop
  cropped_image = image[y_min:y_max, x_min:x_max]

  # crop된 이미지 저장 경로
  output_path = '/content/drive/MyDrive/데청캠_공유/K-Fashion/밀리터리/' + 'cropped_' + file_list_py[i]

  # crop된 이미지 저장
  cv2.imwrite(output_path, cropped_image)

In [ ]:
# 이미지 목록을 불러오기
path = "/content/drive/MyDrive/데청캠/Train/밀리터리/"
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.jpg')]

for i in range(len(file_list_py)):

  # 입력 이미지 로드
  image = Image.open(path + file_list_py[i])
  # 이미지를 Torch Tensor로 변환
  transform = T.Compose([T.ToTensor()])
  input_image = transform(image)
  # 이미지 배치(batch)로 변환
  input_image = input_image.unsqueeze(0)

  # 모델 예측 수행
  with torch.no_grad():
    prediction = model(input_image)

  boxes = prediction[0]['boxes'] # bounding box 좌표
  scores = prediction[0]['scores'] #객체의 확률 값들

  # 확률 값이 가장 큰 bounding box 좌표 출력
  bounding_box = boxes[find_max_index(scores)]

  # 텐서에서 바운딩 박스 좌표 얻기
  x_min, y_min, x_max, y_max = bounding_box

  # 텐서를 정수로 반환(인덱싱 하려고)
  x_min = math.floor(x_min.item())
  y_min = math.floor(y_min.item())
  x_max = math.floor(x_max.item())
  y_max = math.floor(y_max.item())

  # 이미지 읽기
  image = cv2.imread(path + file_list_py[i])

  # 바운딩 박스 좌표를 이용하여 이미지 crop
  cropped_image = image[y_min:y_max, x_min:x_max]

  # crop된 이미지 저장 경로
  output_path = '/content/drive/MyDrive/데청캠_공유/K-Fashion/밀리터리/' + 'cropped_' + file_list_py[i]

  # crop된 이미지 저장
  cv2.imwrite(output_path, cropped_image)

  print(f'{i}번째 이미지가 성공적으로 저장되었습니다.')


0번째 이미지가 성공적으로 저장되었습니다.
1번째 이미지가 성공적으로 저장되었습니다.
2번째 이미지가 성공적으로 저장되었습니다.
3번째 이미지가 성공적으로 저장되었습니다.
4번째 이미지가 성공적으로 저장되었습니다.
5번째 이미지가 성공적으로 저장되었습니다.
6번째 이미지가 성공적으로 저장되었습니다.
7번째 이미지가 성공적으로 저장되었습니다.
8번째 이미지가 성공적으로 저장되었습니다.
9번째 이미지가 성공적으로 저장되었습니다.
10번째 이미지가 성공적으로 저장되었습니다.
11번째 이미지가 성공적으로 저장되었습니다.
12번째 이미지가 성공적으로 저장되었습니다.
13번째 이미지가 성공적으로 저장되었습니다.
14번째 이미지가 성공적으로 저장되었습니다.
15번째 이미지가 성공적으로 저장되었습니다.
16번째 이미지가 성공적으로 저장되었습니다.
17번째 이미지가 성공적으로 저장되었습니다.
18번째 이미지가 성공적으로 저장되었습니다.
19번째 이미지가 성공적으로 저장되었습니다.
20번째 이미지가 성공적으로 저장되었습니다.
21번째 이미지가 성공적으로 저장되었습니다.
22번째 이미지가 성공적으로 저장되었습니다.
23번째 이미지가 성공적으로 저장되었습니다.
24번째 이미지가 성공적으로 저장되었습니다.
25번째 이미지가 성공적으로 저장되었습니다.
26번째 이미지가 성공적으로 저장되었습니다.
27번째 이미지가 성공적으로 저장되었습니다.
28번째 이미지가 성공적으로 저장되었습니다.
29번째 이미지가 성공적으로 저장되었습니다.
30번째 이미지가 성공적으로 저장되었습니다.
31번째 이미지가 성공적으로 저장되었습니다.
32번째 이미지가 성공적으로 저장되었습니다.
33번째 이미지가 성공적으로 저장되었습니다.
34번째 이미지가 성공적으로 저장되었습니다.
35번째 이미지가 성공적으로 저장되었습니다.
36번째 이미지가 성공적으로 저장되었습니다.
37번째 이미지가 성공적으로 저장되었습니다.
38번째 이미지가 성공적으로 저장되었습니다.
39번째 이미지가 성공적으로 저장되었습니다.
40번째 이미지가 

In [ ]:
# 이미지 목록을 불러오기
path = "/content/drive/MyDrive/데청캠/Train/섹시/"
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.jpg')]

for i in range(len(file_list_py)):

  # 입력 이미지 로드
  image = Image.open(path + file_list_py[i])
  # 이미지를 Torch Tensor로 변환
  transform = T.Compose([T.ToTensor()])
  input_image = transform(image)
  # 이미지 배치(batch)로 변환
  input_image = input_image.unsqueeze(0)

  # 모델 예측 수행
  with torch.no_grad():
    prediction = model(input_image)

  boxes = prediction[0]['boxes'] # bounding box 좌표
  scores = prediction[0]['scores'] #객체의 확률 값들

  # 확률 값이 가장 큰 bounding box 좌표 출력
  bounding_box = boxes[find_max_index(scores)]

  # 텐서에서 바운딩 박스 좌표 얻기
  x_min, y_min, x_max, y_max = bounding_box

  # 텐서를 정수로 반환(인덱싱 하려고)
  x_min = math.floor(x_min.item())
  y_min = math.floor(y_min.item())
  x_max = math.floor(x_max.item())
  y_max = math.floor(y_max.item())

  # 이미지 읽기
  image = cv2.imread(path + file_list_py[i])

  # 바운딩 박스 좌표를 이용하여 이미지 crop
  cropped_image = image[y_min:y_max, x_min:x_max]

  # crop된 이미지 저장 경로
  output_path = '/content/drive/MyDrive/데청캠_공유/K-Fashion/섹시/' + 'cropped_' + file_list_py[i]

  # crop된 이미지 저장
  cv2.imwrite(output_path, cropped_image)

  print(f'{i}번째 이미지가 성공적으로 저장되었습니다.')


0번째 이미지가 성공적으로 저장되었습니다.
1번째 이미지가 성공적으로 저장되었습니다.
2번째 이미지가 성공적으로 저장되었습니다.
3번째 이미지가 성공적으로 저장되었습니다.
4번째 이미지가 성공적으로 저장되었습니다.
5번째 이미지가 성공적으로 저장되었습니다.
6번째 이미지가 성공적으로 저장되었습니다.
7번째 이미지가 성공적으로 저장되었습니다.
8번째 이미지가 성공적으로 저장되었습니다.
9번째 이미지가 성공적으로 저장되었습니다.
10번째 이미지가 성공적으로 저장되었습니다.
11번째 이미지가 성공적으로 저장되었습니다.
12번째 이미지가 성공적으로 저장되었습니다.
13번째 이미지가 성공적으로 저장되었습니다.
14번째 이미지가 성공적으로 저장되었습니다.
15번째 이미지가 성공적으로 저장되었습니다.
16번째 이미지가 성공적으로 저장되었습니다.
17번째 이미지가 성공적으로 저장되었습니다.
18번째 이미지가 성공적으로 저장되었습니다.
19번째 이미지가 성공적으로 저장되었습니다.
20번째 이미지가 성공적으로 저장되었습니다.
21번째 이미지가 성공적으로 저장되었습니다.
22번째 이미지가 성공적으로 저장되었습니다.
23번째 이미지가 성공적으로 저장되었습니다.
24번째 이미지가 성공적으로 저장되었습니다.
25번째 이미지가 성공적으로 저장되었습니다.
26번째 이미지가 성공적으로 저장되었습니다.
27번째 이미지가 성공적으로 저장되었습니다.
28번째 이미지가 성공적으로 저장되었습니다.
29번째 이미지가 성공적으로 저장되었습니다.
30번째 이미지가 성공적으로 저장되었습니다.
31번째 이미지가 성공적으로 저장되었습니다.
32번째 이미지가 성공적으로 저장되었습니다.
33번째 이미지가 성공적으로 저장되었습니다.
34번째 이미지가 성공적으로 저장되었습니다.
35번째 이미지가 성공적으로 저장되었습니다.
36번째 이미지가 성공적으로 저장되었습니다.
37번째 이미지가 성공적으로 저장되었습니다.
38번째 이미지가 성공적으로 저장되었습니다.
39번째 이미지가 성공적으로 저장되었습니다.
40번째 이미지가 

In [ ]:
# 이미지 목록을 불러오기
path = "/content/drive/MyDrive/데청캠/Train/소피스트케이티드/"
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.jpg')]

for i in range(len(file_list_py)):

  # 입력 이미지 로드
  image = Image.open(path + file_list_py[i])
  # 이미지를 Torch Tensor로 변환
  transform = T.Compose([T.ToTensor()])
  input_image = transform(image)
  # 이미지 배치(batch)로 변환
  input_image = input_image.unsqueeze(0)

  # 모델 예측 수행
  with torch.no_grad():
    prediction = model(input_image)

  boxes = prediction[0]['boxes'] # bounding box 좌표
  scores = prediction[0]['scores'] #객체의 확률 값들

  # 확률 값이 가장 큰 bounding box 좌표 출력
  bounding_box = boxes[find_max_index(scores)]

  # 텐서에서 바운딩 박스 좌표 얻기
  x_min, y_min, x_max, y_max = bounding_box

  # 텐서를 정수로 반환(인덱싱 하려고)
  x_min = math.floor(x_min.item())
  y_min = math.floor(y_min.item())
  x_max = math.floor(x_max.item())
  y_max = math.floor(y_max.item())

  # 이미지 읽기
  image = cv2.imread(path + file_list_py[i])

  # 바운딩 박스 좌표를 이용하여 이미지 crop
  cropped_image = image[y_min:y_max, x_min:x_max]

  # crop된 이미지 저장 경로
  output_path = '/content/drive/MyDrive/데청캠_공유/K-Fashion/소피스트케이티드/' + 'cropped_' + file_list_py[i]

  # crop된 이미지 저장
  cv2.imwrite(output_path, cropped_image)

  print(f'{i}번째 이미지가 성공적으로 저장되었습니다.')


0번째 이미지가 성공적으로 저장되었습니다.
1번째 이미지가 성공적으로 저장되었습니다.
2번째 이미지가 성공적으로 저장되었습니다.
3번째 이미지가 성공적으로 저장되었습니다.
4번째 이미지가 성공적으로 저장되었습니다.
5번째 이미지가 성공적으로 저장되었습니다.
6번째 이미지가 성공적으로 저장되었습니다.
7번째 이미지가 성공적으로 저장되었습니다.
8번째 이미지가 성공적으로 저장되었습니다.
9번째 이미지가 성공적으로 저장되었습니다.
10번째 이미지가 성공적으로 저장되었습니다.
11번째 이미지가 성공적으로 저장되었습니다.
12번째 이미지가 성공적으로 저장되었습니다.
13번째 이미지가 성공적으로 저장되었습니다.
14번째 이미지가 성공적으로 저장되었습니다.
15번째 이미지가 성공적으로 저장되었습니다.
16번째 이미지가 성공적으로 저장되었습니다.
17번째 이미지가 성공적으로 저장되었습니다.
18번째 이미지가 성공적으로 저장되었습니다.
19번째 이미지가 성공적으로 저장되었습니다.
20번째 이미지가 성공적으로 저장되었습니다.
21번째 이미지가 성공적으로 저장되었습니다.
22번째 이미지가 성공적으로 저장되었습니다.
23번째 이미지가 성공적으로 저장되었습니다.
24번째 이미지가 성공적으로 저장되었습니다.
25번째 이미지가 성공적으로 저장되었습니다.
26번째 이미지가 성공적으로 저장되었습니다.
27번째 이미지가 성공적으로 저장되었습니다.
28번째 이미지가 성공적으로 저장되었습니다.
29번째 이미지가 성공적으로 저장되었습니다.
30번째 이미지가 성공적으로 저장되었습니다.
31번째 이미지가 성공적으로 저장되었습니다.
32번째 이미지가 성공적으로 저장되었습니다.
33번째 이미지가 성공적으로 저장되었습니다.
34번째 이미지가 성공적으로 저장되었습니다.
35번째 이미지가 성공적으로 저장되었습니다.
36번째 이미지가 성공적으로 저장되었습니다.
37번째 이미지가 성공적으로 저장되었습니다.
38번째 이미지가 성공적으로 저장되었습니다.
39번째 이미지가 성공적으로 저장되었습니다.
40번째 이미지가 

In [ ]:
# 이미지 목록을 불러오기
path = "/content/drive/MyDrive/데청캠/Train/스포티/"
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.jpg')]

for i in range(len(file_list_py)):

  # 입력 이미지 로드
  image = Image.open(path + file_list_py[i])
  # 이미지를 Torch Tensor로 변환
  transform = T.Compose([T.ToTensor()])
  input_image = transform(image)
  # 이미지 배치(batch)로 변환
  input_image = input_image.unsqueeze(0)

  # 모델 예측 수행
  with torch.no_grad():
    prediction = model(input_image)

  boxes = prediction[0]['boxes'] # bounding box 좌표
  scores = prediction[0]['scores'] #객체의 확률 값들

  # 확률 값이 가장 큰 bounding box 좌표 출력
  bounding_box = boxes[find_max_index(scores)]

  # 텐서에서 바운딩 박스 좌표 얻기
  x_min, y_min, x_max, y_max = bounding_box

  # 텐서를 정수로 반환(인덱싱 하려고)
  x_min = math.floor(x_min.item())
  y_min = math.floor(y_min.item())
  x_max = math.floor(x_max.item())
  y_max = math.floor(y_max.item())

  # 이미지 읽기
  image = cv2.imread(path + file_list_py[i])

  # 바운딩 박스 좌표를 이용하여 이미지 crop
  cropped_image = image[y_min:y_max, x_min:x_max]

  # crop된 이미지 저장 경로
  output_path = '/content/drive/MyDrive/데청캠_공유/K-Fashion/스포티/' + 'cropped_' + file_list_py[i]

  # crop된 이미지 저장
  cv2.imwrite(output_path, cropped_image)

  print(f'{i}번째 이미지가 성공적으로 저장되었습니다.')


0번째 이미지가 성공적으로 저장되었습니다.
1번째 이미지가 성공적으로 저장되었습니다.
2번째 이미지가 성공적으로 저장되었습니다.
3번째 이미지가 성공적으로 저장되었습니다.
4번째 이미지가 성공적으로 저장되었습니다.
5번째 이미지가 성공적으로 저장되었습니다.
6번째 이미지가 성공적으로 저장되었습니다.
7번째 이미지가 성공적으로 저장되었습니다.
8번째 이미지가 성공적으로 저장되었습니다.
9번째 이미지가 성공적으로 저장되었습니다.
10번째 이미지가 성공적으로 저장되었습니다.
11번째 이미지가 성공적으로 저장되었습니다.
12번째 이미지가 성공적으로 저장되었습니다.
13번째 이미지가 성공적으로 저장되었습니다.
14번째 이미지가 성공적으로 저장되었습니다.
15번째 이미지가 성공적으로 저장되었습니다.
16번째 이미지가 성공적으로 저장되었습니다.
17번째 이미지가 성공적으로 저장되었습니다.
18번째 이미지가 성공적으로 저장되었습니다.
19번째 이미지가 성공적으로 저장되었습니다.
20번째 이미지가 성공적으로 저장되었습니다.
21번째 이미지가 성공적으로 저장되었습니다.
22번째 이미지가 성공적으로 저장되었습니다.
23번째 이미지가 성공적으로 저장되었습니다.
24번째 이미지가 성공적으로 저장되었습니다.
25번째 이미지가 성공적으로 저장되었습니다.
26번째 이미지가 성공적으로 저장되었습니다.
27번째 이미지가 성공적으로 저장되었습니다.
28번째 이미지가 성공적으로 저장되었습니다.
29번째 이미지가 성공적으로 저장되었습니다.
30번째 이미지가 성공적으로 저장되었습니다.
31번째 이미지가 성공적으로 저장되었습니다.
32번째 이미지가 성공적으로 저장되었습니다.
33번째 이미지가 성공적으로 저장되었습니다.
34번째 이미지가 성공적으로 저장되었습니다.
35번째 이미지가 성공적으로 저장되었습니다.
36번째 이미지가 성공적으로 저장되었습니다.
37번째 이미지가 성공적으로 저장되었습니다.
38번째 이미지가 성공적으로 저장되었습니다.
39번째 이미지가 성공적으로 저장되었습니다.
40번째 이미지가 

In [ ]:
# 이미지 목록을 불러오기
path = "/content/drive/MyDrive/데청캠/Train/아방가르드/"
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.jpg')]

for i in range(len(file_list_py)):

  # 입력 이미지 로드
  image = Image.open(path + file_list_py[i])
  # 이미지를 Torch Tensor로 변환
  transform = T.Compose([T.ToTensor()])
  input_image = transform(image)
  # 이미지 배치(batch)로 변환
  input_image = input_image.unsqueeze(0)

  # 모델 예측 수행
  with torch.no_grad():
    prediction = model(input_image)

  boxes = prediction[0]['boxes'] # bounding box 좌표
  scores = prediction[0]['scores'] #객체의 확률 값들

  # 확률 값이 가장 큰 bounding box 좌표 출력
  bounding_box = boxes[find_max_index(scores)]

  # 텐서에서 바운딩 박스 좌표 얻기
  x_min, y_min, x_max, y_max = bounding_box

  # 텐서를 정수로 반환(인덱싱 하려고)
  x_min = math.floor(x_min.item())
  y_min = math.floor(y_min.item())
  x_max = math.floor(x_max.item())
  y_max = math.floor(y_max.item())

  # 이미지 읽기
  image = cv2.imread(path + file_list_py[i])

  # 바운딩 박스 좌표를 이용하여 이미지 crop
  cropped_image = image[y_min:y_max, x_min:x_max]

  # crop된 이미지 저장 경로
  output_path = '/content/drive/MyDrive/데청캠_공유/K-Fashion/아방가르드/' + 'cropped_' + file_list_py[i]

  # crop된 이미지 저장
  cv2.imwrite(output_path, cropped_image)

  print(f'{i}번째 이미지가 성공적으로 저장되었습니다.')


0번째 이미지가 성공적으로 저장되었습니다.
1번째 이미지가 성공적으로 저장되었습니다.
2번째 이미지가 성공적으로 저장되었습니다.
3번째 이미지가 성공적으로 저장되었습니다.
4번째 이미지가 성공적으로 저장되었습니다.
5번째 이미지가 성공적으로 저장되었습니다.
6번째 이미지가 성공적으로 저장되었습니다.
7번째 이미지가 성공적으로 저장되었습니다.
8번째 이미지가 성공적으로 저장되었습니다.
9번째 이미지가 성공적으로 저장되었습니다.
10번째 이미지가 성공적으로 저장되었습니다.
11번째 이미지가 성공적으로 저장되었습니다.
12번째 이미지가 성공적으로 저장되었습니다.
13번째 이미지가 성공적으로 저장되었습니다.
14번째 이미지가 성공적으로 저장되었습니다.
15번째 이미지가 성공적으로 저장되었습니다.
16번째 이미지가 성공적으로 저장되었습니다.
17번째 이미지가 성공적으로 저장되었습니다.
18번째 이미지가 성공적으로 저장되었습니다.
19번째 이미지가 성공적으로 저장되었습니다.
20번째 이미지가 성공적으로 저장되었습니다.
21번째 이미지가 성공적으로 저장되었습니다.
22번째 이미지가 성공적으로 저장되었습니다.
23번째 이미지가 성공적으로 저장되었습니다.
24번째 이미지가 성공적으로 저장되었습니다.
25번째 이미지가 성공적으로 저장되었습니다.
26번째 이미지가 성공적으로 저장되었습니다.
27번째 이미지가 성공적으로 저장되었습니다.
28번째 이미지가 성공적으로 저장되었습니다.
29번째 이미지가 성공적으로 저장되었습니다.
30번째 이미지가 성공적으로 저장되었습니다.
31번째 이미지가 성공적으로 저장되었습니다.
32번째 이미지가 성공적으로 저장되었습니다.
33번째 이미지가 성공적으로 저장되었습니다.
34번째 이미지가 성공적으로 저장되었습니다.
35번째 이미지가 성공적으로 저장되었습니다.
36번째 이미지가 성공적으로 저장되었습니다.
37번째 이미지가 성공적으로 저장되었습니다.
38번째 이미지가 성공적으로 저장되었습니다.
39번째 이미지가 성공적으로 저장되었습니다.
40번째 이미지가 

In [ ]:
# 이미지 목록을 불러오기
path = "/content/drive/MyDrive/데청캠/Train/오리엔탈/"
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.jpg')]

for i in range(len(file_list_py)):

  # 입력 이미지 로드
  image = Image.open(path + file_list_py[i])
  # 이미지를 Torch Tensor로 변환
  transform = T.Compose([T.ToTensor()])
  input_image = transform(image)
  # 이미지 배치(batch)로 변환
  input_image = input_image.unsqueeze(0)

  # 모델 예측 수행
  with torch.no_grad():
    prediction = model(input_image)

  boxes = prediction[0]['boxes'] # bounding box 좌표
  scores = prediction[0]['scores'] #객체의 확률 값들

  # 확률 값이 가장 큰 bounding box 좌표 출력
  bounding_box = boxes[find_max_index(scores)]

  # 텐서에서 바운딩 박스 좌표 얻기
  x_min, y_min, x_max, y_max = bounding_box

  # 텐서를 정수로 반환(인덱싱 하려고)
  x_min = math.floor(x_min.item())
  y_min = math.floor(y_min.item())
  x_max = math.floor(x_max.item())
  y_max = math.floor(y_max.item())

  # 이미지 읽기
  image = cv2.imread(path + file_list_py[i])

  # 바운딩 박스 좌표를 이용하여 이미지 crop
  cropped_image = image[y_min:y_max, x_min:x_max]

  # crop된 이미지 저장 경로
  output_path = '/content/drive/MyDrive/데청캠_공유/K-Fashion/오리엔탈/' + 'cropped_' + file_list_py[i]

  # crop된 이미지 저장
  cv2.imwrite(output_path, cropped_image)

  print(f'{i}번째 이미지가 성공적으로 저장되었습니다.')


0번째 이미지가 성공적으로 저장되었습니다.
1번째 이미지가 성공적으로 저장되었습니다.
2번째 이미지가 성공적으로 저장되었습니다.
3번째 이미지가 성공적으로 저장되었습니다.
4번째 이미지가 성공적으로 저장되었습니다.
5번째 이미지가 성공적으로 저장되었습니다.
6번째 이미지가 성공적으로 저장되었습니다.
7번째 이미지가 성공적으로 저장되었습니다.
8번째 이미지가 성공적으로 저장되었습니다.
9번째 이미지가 성공적으로 저장되었습니다.
10번째 이미지가 성공적으로 저장되었습니다.
11번째 이미지가 성공적으로 저장되었습니다.
12번째 이미지가 성공적으로 저장되었습니다.
13번째 이미지가 성공적으로 저장되었습니다.
14번째 이미지가 성공적으로 저장되었습니다.
15번째 이미지가 성공적으로 저장되었습니다.
16번째 이미지가 성공적으로 저장되었습니다.
17번째 이미지가 성공적으로 저장되었습니다.
18번째 이미지가 성공적으로 저장되었습니다.
19번째 이미지가 성공적으로 저장되었습니다.
20번째 이미지가 성공적으로 저장되었습니다.
21번째 이미지가 성공적으로 저장되었습니다.
22번째 이미지가 성공적으로 저장되었습니다.
23번째 이미지가 성공적으로 저장되었습니다.
24번째 이미지가 성공적으로 저장되었습니다.
25번째 이미지가 성공적으로 저장되었습니다.
26번째 이미지가 성공적으로 저장되었습니다.
27번째 이미지가 성공적으로 저장되었습니다.
28번째 이미지가 성공적으로 저장되었습니다.
29번째 이미지가 성공적으로 저장되었습니다.
30번째 이미지가 성공적으로 저장되었습니다.
31번째 이미지가 성공적으로 저장되었습니다.
32번째 이미지가 성공적으로 저장되었습니다.
33번째 이미지가 성공적으로 저장되었습니다.
34번째 이미지가 성공적으로 저장되었습니다.
35번째 이미지가 성공적으로 저장되었습니다.
36번째 이미지가 성공적으로 저장되었습니다.
37번째 이미지가 성공적으로 저장되었습니다.
38번째 이미지가 성공적으로 저장되었습니다.
39번째 이미지가 성공적으로 저장되었습니다.
40번째 이미지가 

In [ ]:
# 이미지 목록을 불러오기
path = "/content/drive/MyDrive/데청캠/Train/웨스턴/"
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.jpg')]

for i in range(len(file_list_py)):

  # 입력 이미지 로드
  image = Image.open(path + file_list_py[i])
  # 이미지를 Torch Tensor로 변환
  transform = T.Compose([T.ToTensor()])
  input_image = transform(image)
  # 이미지 배치(batch)로 변환
  input_image = input_image.unsqueeze(0)

  # 모델 예측 수행
  with torch.no_grad():
    prediction = model(input_image)

  boxes = prediction[0]['boxes'] # bounding box 좌표
  scores = prediction[0]['scores'] #객체의 확률 값들

  # 확률 값이 가장 큰 bounding box 좌표 출력
  bounding_box = boxes[find_max_index(scores)]

  # 텐서에서 바운딩 박스 좌표 얻기
  x_min, y_min, x_max, y_max = bounding_box

  # 텐서를 정수로 반환(인덱싱 하려고)
  x_min = math.floor(x_min.item())
  y_min = math.floor(y_min.item())
  x_max = math.floor(x_max.item())
  y_max = math.floor(y_max.item())

  # 이미지 읽기
  image = cv2.imread(path + file_list_py[i])

  # 바운딩 박스 좌표를 이용하여 이미지 crop
  cropped_image = image[y_min:y_max, x_min:x_max]

  # crop된 이미지 저장 경로
  output_path = '/content/drive/MyDrive/데청캠_공유/K-Fashion/웨스턴/' + 'cropped_' + file_list_py[i]

  # crop된 이미지 저장
  cv2.imwrite(output_path, cropped_image)

  print(f'{i}번째 이미지가 성공적으로 저장되었습니다.')


0번째 이미지가 성공적으로 저장되었습니다.
1번째 이미지가 성공적으로 저장되었습니다.
2번째 이미지가 성공적으로 저장되었습니다.
3번째 이미지가 성공적으로 저장되었습니다.
4번째 이미지가 성공적으로 저장되었습니다.
5번째 이미지가 성공적으로 저장되었습니다.
6번째 이미지가 성공적으로 저장되었습니다.
7번째 이미지가 성공적으로 저장되었습니다.
8번째 이미지가 성공적으로 저장되었습니다.
9번째 이미지가 성공적으로 저장되었습니다.
10번째 이미지가 성공적으로 저장되었습니다.
11번째 이미지가 성공적으로 저장되었습니다.
12번째 이미지가 성공적으로 저장되었습니다.
13번째 이미지가 성공적으로 저장되었습니다.
14번째 이미지가 성공적으로 저장되었습니다.
15번째 이미지가 성공적으로 저장되었습니다.
16번째 이미지가 성공적으로 저장되었습니다.
17번째 이미지가 성공적으로 저장되었습니다.
18번째 이미지가 성공적으로 저장되었습니다.
19번째 이미지가 성공적으로 저장되었습니다.
20번째 이미지가 성공적으로 저장되었습니다.
21번째 이미지가 성공적으로 저장되었습니다.
22번째 이미지가 성공적으로 저장되었습니다.
23번째 이미지가 성공적으로 저장되었습니다.
24번째 이미지가 성공적으로 저장되었습니다.
25번째 이미지가 성공적으로 저장되었습니다.
26번째 이미지가 성공적으로 저장되었습니다.
27번째 이미지가 성공적으로 저장되었습니다.
28번째 이미지가 성공적으로 저장되었습니다.
29번째 이미지가 성공적으로 저장되었습니다.
30번째 이미지가 성공적으로 저장되었습니다.
31번째 이미지가 성공적으로 저장되었습니다.
32번째 이미지가 성공적으로 저장되었습니다.
33번째 이미지가 성공적으로 저장되었습니다.
34번째 이미지가 성공적으로 저장되었습니다.
35번째 이미지가 성공적으로 저장되었습니다.
36번째 이미지가 성공적으로 저장되었습니다.
37번째 이미지가 성공적으로 저장되었습니다.
38번째 이미지가 성공적으로 저장되었습니다.
39번째 이미지가 성공적으로 저장되었습니다.
40번째 이미지가 

In [ ]:
# 이미지 목록을 불러오기
path = "/content/drive/MyDrive/데청캠/Train/젠더리스/"
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.jpg')]

for i in range(len(file_list_py)):

  # 입력 이미지 로드
  image = Image.open(path + file_list_py[i])
  # 이미지를 Torch Tensor로 변환
  transform = T.Compose([T.ToTensor()])
  input_image = transform(image)
  # 이미지 배치(batch)로 변환
  input_image = input_image.unsqueeze(0)

  # 모델 예측 수행
  with torch.no_grad():
    prediction = model(input_image)

  boxes = prediction[0]['boxes'] # bounding box 좌표
  scores = prediction[0]['scores'] #객체의 확률 값들

  # 확률 값이 가장 큰 bounding box 좌표 출력
  bounding_box = boxes[find_max_index(scores)]

  # 텐서에서 바운딩 박스 좌표 얻기
  x_min, y_min, x_max, y_max = bounding_box

  # 텐서를 정수로 반환(인덱싱 하려고)
  x_min = math.floor(x_min.item())
  y_min = math.floor(y_min.item())
  x_max = math.floor(x_max.item())
  y_max = math.floor(y_max.item())

  # 이미지 읽기
  image = cv2.imread(path + file_list_py[i])

  # 바운딩 박스 좌표를 이용하여 이미지 crop
  cropped_image = image[y_min:y_max, x_min:x_max]

  # crop된 이미지 저장 경로
  output_path = '/content/drive/MyDrive/데청캠_공유/K-Fashion/젠더리스/' + 'cropped_' + file_list_py[i]

  # crop된 이미지 저장
  cv2.imwrite(output_path, cropped_image)

  print(f'{i}번째 이미지가 성공적으로 저장되었습니다.')


0번째 이미지가 성공적으로 저장되었습니다.
1번째 이미지가 성공적으로 저장되었습니다.
2번째 이미지가 성공적으로 저장되었습니다.
3번째 이미지가 성공적으로 저장되었습니다.
4번째 이미지가 성공적으로 저장되었습니다.
5번째 이미지가 성공적으로 저장되었습니다.
6번째 이미지가 성공적으로 저장되었습니다.
7번째 이미지가 성공적으로 저장되었습니다.
8번째 이미지가 성공적으로 저장되었습니다.
9번째 이미지가 성공적으로 저장되었습니다.
10번째 이미지가 성공적으로 저장되었습니다.
11번째 이미지가 성공적으로 저장되었습니다.
12번째 이미지가 성공적으로 저장되었습니다.
13번째 이미지가 성공적으로 저장되었습니다.
14번째 이미지가 성공적으로 저장되었습니다.
15번째 이미지가 성공적으로 저장되었습니다.
16번째 이미지가 성공적으로 저장되었습니다.
17번째 이미지가 성공적으로 저장되었습니다.
18번째 이미지가 성공적으로 저장되었습니다.
19번째 이미지가 성공적으로 저장되었습니다.
20번째 이미지가 성공적으로 저장되었습니다.
21번째 이미지가 성공적으로 저장되었습니다.
22번째 이미지가 성공적으로 저장되었습니다.
23번째 이미지가 성공적으로 저장되었습니다.
24번째 이미지가 성공적으로 저장되었습니다.
25번째 이미지가 성공적으로 저장되었습니다.
26번째 이미지가 성공적으로 저장되었습니다.
27번째 이미지가 성공적으로 저장되었습니다.
28번째 이미지가 성공적으로 저장되었습니다.
29번째 이미지가 성공적으로 저장되었습니다.
30번째 이미지가 성공적으로 저장되었습니다.
31번째 이미지가 성공적으로 저장되었습니다.
32번째 이미지가 성공적으로 저장되었습니다.
33번째 이미지가 성공적으로 저장되었습니다.
34번째 이미지가 성공적으로 저장되었습니다.
35번째 이미지가 성공적으로 저장되었습니다.
36번째 이미지가 성공적으로 저장되었습니다.
37번째 이미지가 성공적으로 저장되었습니다.
38번째 이미지가 성공적으로 저장되었습니다.
39번째 이미지가 성공적으로 저장되었습니다.
40번째 이미지가 

In [ ]:
# 이미지 목록을 불러오기
path = "/content/drive/MyDrive/데청캠/Train/컨트리/"
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.jpg')]

for i in range(78,len(file_list_py)):

  # 입력 이미지 로드
  image = Image.open(path + file_list_py[i])
  # 이미지를 Torch Tensor로 변환
  transform = T.Compose([T.ToTensor()])
  input_image = transform(image)
  # 이미지 배치(batch)로 변환
  input_image = input_image.unsqueeze(0)

  # 모델 예측 수행
  with torch.no_grad():
    prediction = model(input_image)

  boxes = prediction[0]['boxes'] # bounding box 좌표
  scores = prediction[0]['scores'] #객체의 확률 값들

  # 확률 값이 가장 큰 bounding box 좌표 출력
  bounding_box = boxes[find_max_index(scores)]

  # 텐서에서 바운딩 박스 좌표 얻기
  x_min, y_min, x_max, y_max = bounding_box

  # 텐서를 정수로 반환(인덱싱 하려고)
  x_min = math.floor(x_min.item())
  y_min = math.floor(y_min.item())
  x_max = math.floor(x_max.item())
  y_max = math.floor(y_max.item())

  # 이미지 읽기
  image = cv2.imread(path + file_list_py[i])

  # 바운딩 박스 좌표를 이용하여 이미지 crop
  cropped_image = image[y_min:y_max, x_min:x_max]

  # crop된 이미지 저장 경로
  output_path = '/content/drive/MyDrive/데청캠_공유/K-Fashion/컨트리/' + 'cropped_' + file_list_py[i]

  # crop된 이미지 저장
  cv2.imwrite(output_path, cropped_image)

  print(f'{i}번째 이미지가 성공적으로 저장되었습니다.')


78번째 이미지가 성공적으로 저장되었습니다.
79번째 이미지가 성공적으로 저장되었습니다.
80번째 이미지가 성공적으로 저장되었습니다.
81번째 이미지가 성공적으로 저장되었습니다.
82번째 이미지가 성공적으로 저장되었습니다.
83번째 이미지가 성공적으로 저장되었습니다.
84번째 이미지가 성공적으로 저장되었습니다.
85번째 이미지가 성공적으로 저장되었습니다.
86번째 이미지가 성공적으로 저장되었습니다.
87번째 이미지가 성공적으로 저장되었습니다.
88번째 이미지가 성공적으로 저장되었습니다.
89번째 이미지가 성공적으로 저장되었습니다.
90번째 이미지가 성공적으로 저장되었습니다.
91번째 이미지가 성공적으로 저장되었습니다.
92번째 이미지가 성공적으로 저장되었습니다.
93번째 이미지가 성공적으로 저장되었습니다.
94번째 이미지가 성공적으로 저장되었습니다.
95번째 이미지가 성공적으로 저장되었습니다.
96번째 이미지가 성공적으로 저장되었습니다.
97번째 이미지가 성공적으로 저장되었습니다.
98번째 이미지가 성공적으로 저장되었습니다.
99번째 이미지가 성공적으로 저장되었습니다.
100번째 이미지가 성공적으로 저장되었습니다.
101번째 이미지가 성공적으로 저장되었습니다.
102번째 이미지가 성공적으로 저장되었습니다.
103번째 이미지가 성공적으로 저장되었습니다.
104번째 이미지가 성공적으로 저장되었습니다.
105번째 이미지가 성공적으로 저장되었습니다.
106번째 이미지가 성공적으로 저장되었습니다.
107번째 이미지가 성공적으로 저장되었습니다.
108번째 이미지가 성공적으로 저장되었습니다.
109번째 이미지가 성공적으로 저장되었습니다.
110번째 이미지가 성공적으로 저장되었습니다.
111번째 이미지가 성공적으로 저장되었습니다.
112번째 이미지가 성공적으로 저장되었습니다.
113번째 이미지가 성공적으로 저장되었습니다.
114번째 이미지가 성공적으로 저장되었습니다.
115번째 이미지가 성공적으로 저장되었습니다.
116번째 이미지가 성공적으로 저장되었습니다.
117번째 이미

In [ ]:
# 이미지 목록을 불러오기
path = "/content/drive/MyDrive/데청캠/Train/클래식/"
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.jpg')]

for i in range(len(file_list_py)):

  # 입력 이미지 로드
  image = Image.open(path + file_list_py[i])
  # 이미지를 Torch Tensor로 변환
  transform = T.Compose([T.ToTensor()])
  input_image = transform(image)
  # 이미지 배치(batch)로 변환
  input_image = input_image.unsqueeze(0)

  # 모델 예측 수행
  with torch.no_grad():
    prediction = model(input_image)

  boxes = prediction[0]['boxes'] # bounding box 좌표
  scores = prediction[0]['scores'] #객체의 확률 값들

  # 확률 값이 가장 큰 bounding box 좌표 출력
  bounding_box = boxes[find_max_index(scores)]

  # 텐서에서 바운딩 박스 좌표 얻기
  x_min, y_min, x_max, y_max = bounding_box

  # 텐서를 정수로 반환(인덱싱 하려고)
  x_min = math.floor(x_min.item())
  y_min = math.floor(y_min.item())
  x_max = math.floor(x_max.item())
  y_max = math.floor(y_max.item())

  # 이미지 읽기
  image = cv2.imread(path + file_list_py[i])

  # 바운딩 박스 좌표를 이용하여 이미지 crop
  cropped_image = image[y_min:y_max, x_min:x_max]

  # crop된 이미지 저장 경로
  output_path = '/content/drive/MyDrive/데청캠_공유/K-Fashion/클래식/' + 'cropped_' + file_list_py[i]

  # crop된 이미지 저장
  cv2.imwrite(output_path, cropped_image)

  print(f'{i}번째 이미지가 성공적으로 저장되었습니다.')


0번째 이미지가 성공적으로 저장되었습니다.
1번째 이미지가 성공적으로 저장되었습니다.
2번째 이미지가 성공적으로 저장되었습니다.
3번째 이미지가 성공적으로 저장되었습니다.
4번째 이미지가 성공적으로 저장되었습니다.
5번째 이미지가 성공적으로 저장되었습니다.
6번째 이미지가 성공적으로 저장되었습니다.
7번째 이미지가 성공적으로 저장되었습니다.
8번째 이미지가 성공적으로 저장되었습니다.
9번째 이미지가 성공적으로 저장되었습니다.
10번째 이미지가 성공적으로 저장되었습니다.
11번째 이미지가 성공적으로 저장되었습니다.
12번째 이미지가 성공적으로 저장되었습니다.
13번째 이미지가 성공적으로 저장되었습니다.
14번째 이미지가 성공적으로 저장되었습니다.
15번째 이미지가 성공적으로 저장되었습니다.
16번째 이미지가 성공적으로 저장되었습니다.
17번째 이미지가 성공적으로 저장되었습니다.
18번째 이미지가 성공적으로 저장되었습니다.
19번째 이미지가 성공적으로 저장되었습니다.
20번째 이미지가 성공적으로 저장되었습니다.
21번째 이미지가 성공적으로 저장되었습니다.
22번째 이미지가 성공적으로 저장되었습니다.
23번째 이미지가 성공적으로 저장되었습니다.
24번째 이미지가 성공적으로 저장되었습니다.
25번째 이미지가 성공적으로 저장되었습니다.
26번째 이미지가 성공적으로 저장되었습니다.
27번째 이미지가 성공적으로 저장되었습니다.
28번째 이미지가 성공적으로 저장되었습니다.
29번째 이미지가 성공적으로 저장되었습니다.
30번째 이미지가 성공적으로 저장되었습니다.
31번째 이미지가 성공적으로 저장되었습니다.
32번째 이미지가 성공적으로 저장되었습니다.
33번째 이미지가 성공적으로 저장되었습니다.
34번째 이미지가 성공적으로 저장되었습니다.
35번째 이미지가 성공적으로 저장되었습니다.
36번째 이미지가 성공적으로 저장되었습니다.
37번째 이미지가 성공적으로 저장되었습니다.
38번째 이미지가 성공적으로 저장되었습니다.
39번째 이미지가 성공적으로 저장되었습니다.
40번째 이미지가 

In [ ]:
# 이미지 목록을 불러오기
path = "/content/drive/MyDrive/데청캠/Train/키치/"
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.jpg')]

for i in range(len(file_list_py)):

  # 입력 이미지 로드
  image = Image.open(path + file_list_py[i])
  # 이미지를 Torch Tensor로 변환
  transform = T.Compose([T.ToTensor()])
  input_image = transform(image)
  # 이미지 배치(batch)로 변환
  input_image = input_image.unsqueeze(0)

  # 모델 예측 수행
  with torch.no_grad():
    prediction = model(input_image)

  boxes = prediction[0]['boxes'] # bounding box 좌표
  scores = prediction[0]['scores'] #객체의 확률 값들

  # 확률 값이 가장 큰 bounding box 좌표 출력
  bounding_box = boxes[find_max_index(scores)]

  # 텐서에서 바운딩 박스 좌표 얻기
  x_min, y_min, x_max, y_max = bounding_box

  # 텐서를 정수로 반환(인덱싱 하려고)
  x_min = math.floor(x_min.item())
  y_min = math.floor(y_min.item())
  x_max = math.floor(x_max.item())
  y_max = math.floor(y_max.item())

  # 이미지 읽기
  image = cv2.imread(path + file_list_py[i])

  # 바운딩 박스 좌표를 이용하여 이미지 crop
  cropped_image = image[y_min:y_max, x_min:x_max]

  # crop된 이미지 저장 경로
  output_path = '/content/drive/MyDrive/데청캠_공유/K-Fashion/키치/' + 'cropped_' + file_list_py[i]

  # crop된 이미지 저장
  cv2.imwrite(output_path, cropped_image)

  print(f'{i}번째 이미지가 성공적으로 저장되었습니다.')


0번째 이미지가 성공적으로 저장되었습니다.
1번째 이미지가 성공적으로 저장되었습니다.
2번째 이미지가 성공적으로 저장되었습니다.
3번째 이미지가 성공적으로 저장되었습니다.
4번째 이미지가 성공적으로 저장되었습니다.
5번째 이미지가 성공적으로 저장되었습니다.
6번째 이미지가 성공적으로 저장되었습니다.
7번째 이미지가 성공적으로 저장되었습니다.
8번째 이미지가 성공적으로 저장되었습니다.
9번째 이미지가 성공적으로 저장되었습니다.
10번째 이미지가 성공적으로 저장되었습니다.
11번째 이미지가 성공적으로 저장되었습니다.
12번째 이미지가 성공적으로 저장되었습니다.
13번째 이미지가 성공적으로 저장되었습니다.
14번째 이미지가 성공적으로 저장되었습니다.
15번째 이미지가 성공적으로 저장되었습니다.
16번째 이미지가 성공적으로 저장되었습니다.
17번째 이미지가 성공적으로 저장되었습니다.
18번째 이미지가 성공적으로 저장되었습니다.
19번째 이미지가 성공적으로 저장되었습니다.
20번째 이미지가 성공적으로 저장되었습니다.
21번째 이미지가 성공적으로 저장되었습니다.
22번째 이미지가 성공적으로 저장되었습니다.
23번째 이미지가 성공적으로 저장되었습니다.
24번째 이미지가 성공적으로 저장되었습니다.
25번째 이미지가 성공적으로 저장되었습니다.
26번째 이미지가 성공적으로 저장되었습니다.
27번째 이미지가 성공적으로 저장되었습니다.
28번째 이미지가 성공적으로 저장되었습니다.
29번째 이미지가 성공적으로 저장되었습니다.
30번째 이미지가 성공적으로 저장되었습니다.
31번째 이미지가 성공적으로 저장되었습니다.
32번째 이미지가 성공적으로 저장되었습니다.
33번째 이미지가 성공적으로 저장되었습니다.
34번째 이미지가 성공적으로 저장되었습니다.
35번째 이미지가 성공적으로 저장되었습니다.
36번째 이미지가 성공적으로 저장되었습니다.
37번째 이미지가 성공적으로 저장되었습니다.
38번째 이미지가 성공적으로 저장되었습니다.
39번째 이미지가 성공적으로 저장되었습니다.
40번째 이미지가 

In [ ]:
# 이미지 목록을 불러오기
path = "/content/drive/MyDrive/데청캠/Train/톰보이/"
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.jpg')]

for i in range(len(file_list_py)):

  # 입력 이미지 로드
  image = Image.open(path + file_list_py[i])
  # 이미지를 Torch Tensor로 변환
  transform = T.Compose([T.ToTensor()])
  input_image = transform(image)
  # 이미지 배치(batch)로 변환
  input_image = input_image.unsqueeze(0)

  # 모델 예측 수행
  with torch.no_grad():
    prediction = model(input_image)

  boxes = prediction[0]['boxes'] # bounding box 좌표
  scores = prediction[0]['scores'] #객체의 확률 값들

  # 확률 값이 가장 큰 bounding box 좌표 출력
  bounding_box = boxes[find_max_index(scores)]

  # 텐서에서 바운딩 박스 좌표 얻기
  x_min, y_min, x_max, y_max = bounding_box

  # 텐서를 정수로 반환(인덱싱 하려고)
  x_min = math.floor(x_min.item())
  y_min = math.floor(y_min.item())
  x_max = math.floor(x_max.item())
  y_max = math.floor(y_max.item())

  # 이미지 읽기
  image = cv2.imread(path + file_list_py[i])

  # 바운딩 박스 좌표를 이용하여 이미지 crop
  cropped_image = image[y_min:y_max, x_min:x_max]

  # crop된 이미지 저장 경로
  output_path = '/content/drive/MyDrive/데청캠_공유/K-Fashion/톰보이/' + 'cropped_' + file_list_py[i]

  # crop된 이미지 저장
  cv2.imwrite(output_path, cropped_image)

  print(f'{i}번째 이미지가 성공적으로 저장되었습니다.')


0번째 이미지가 성공적으로 저장되었습니다.
1번째 이미지가 성공적으로 저장되었습니다.
2번째 이미지가 성공적으로 저장되었습니다.
3번째 이미지가 성공적으로 저장되었습니다.
4번째 이미지가 성공적으로 저장되었습니다.
5번째 이미지가 성공적으로 저장되었습니다.
6번째 이미지가 성공적으로 저장되었습니다.
7번째 이미지가 성공적으로 저장되었습니다.
8번째 이미지가 성공적으로 저장되었습니다.
9번째 이미지가 성공적으로 저장되었습니다.
10번째 이미지가 성공적으로 저장되었습니다.
11번째 이미지가 성공적으로 저장되었습니다.
12번째 이미지가 성공적으로 저장되었습니다.
13번째 이미지가 성공적으로 저장되었습니다.
14번째 이미지가 성공적으로 저장되었습니다.
15번째 이미지가 성공적으로 저장되었습니다.
16번째 이미지가 성공적으로 저장되었습니다.
17번째 이미지가 성공적으로 저장되었습니다.
18번째 이미지가 성공적으로 저장되었습니다.
19번째 이미지가 성공적으로 저장되었습니다.
20번째 이미지가 성공적으로 저장되었습니다.
21번째 이미지가 성공적으로 저장되었습니다.
22번째 이미지가 성공적으로 저장되었습니다.
23번째 이미지가 성공적으로 저장되었습니다.
24번째 이미지가 성공적으로 저장되었습니다.
25번째 이미지가 성공적으로 저장되었습니다.
26번째 이미지가 성공적으로 저장되었습니다.
27번째 이미지가 성공적으로 저장되었습니다.
28번째 이미지가 성공적으로 저장되었습니다.
29번째 이미지가 성공적으로 저장되었습니다.
30번째 이미지가 성공적으로 저장되었습니다.
31번째 이미지가 성공적으로 저장되었습니다.
32번째 이미지가 성공적으로 저장되었습니다.
33번째 이미지가 성공적으로 저장되었습니다.
34번째 이미지가 성공적으로 저장되었습니다.
35번째 이미지가 성공적으로 저장되었습니다.
36번째 이미지가 성공적으로 저장되었습니다.
37번째 이미지가 성공적으로 저장되었습니다.
38번째 이미지가 성공적으로 저장되었습니다.
39번째 이미지가 성공적으로 저장되었습니다.
40번째 이미지가 

In [ ]:
# 이미지 목록을 불러오기
path = "/content/drive/MyDrive/데청캠/Train/펑크/"
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.jpg')]

for i in range(len(file_list_py)):

  # 입력 이미지 로드
  image = Image.open(path + file_list_py[i])
  # 이미지를 Torch Tensor로 변환
  transform = T.Compose([T.ToTensor()])
  input_image = transform(image)
  # 이미지 배치(batch)로 변환
  input_image = input_image.unsqueeze(0)

  # 모델 예측 수행
  with torch.no_grad():
    prediction = model(input_image)

  boxes = prediction[0]['boxes'] # bounding box 좌표
  scores = prediction[0]['scores'] #객체의 확률 값들

  # 확률 값이 가장 큰 bounding box 좌표 출력
  bounding_box = boxes[find_max_index(scores)]

  # 텐서에서 바운딩 박스 좌표 얻기
  x_min, y_min, x_max, y_max = bounding_box

  # 텐서를 정수로 반환(인덱싱 하려고)
  x_min = math.floor(x_min.item())
  y_min = math.floor(y_min.item())
  x_max = math.floor(x_max.item())
  y_max = math.floor(y_max.item())

  # 이미지 읽기
  image = cv2.imread(path + file_list_py[i])

  # 바운딩 박스 좌표를 이용하여 이미지 crop
  cropped_image = image[y_min:y_max, x_min:x_max]

  # crop된 이미지 저장 경로
  output_path = '/content/drive/MyDrive/데청캠_공유/K-Fashion/펑크/' + 'cropped_' + file_list_py[i]

  # crop된 이미지 저장
  cv2.imwrite(output_path, cropped_image)

  print(f'{i}번째 이미지가 성공적으로 저장되었습니다.')


0번째 이미지가 성공적으로 저장되었습니다.
1번째 이미지가 성공적으로 저장되었습니다.
2번째 이미지가 성공적으로 저장되었습니다.
3번째 이미지가 성공적으로 저장되었습니다.
4번째 이미지가 성공적으로 저장되었습니다.
5번째 이미지가 성공적으로 저장되었습니다.
6번째 이미지가 성공적으로 저장되었습니다.
7번째 이미지가 성공적으로 저장되었습니다.
8번째 이미지가 성공적으로 저장되었습니다.
9번째 이미지가 성공적으로 저장되었습니다.
10번째 이미지가 성공적으로 저장되었습니다.
11번째 이미지가 성공적으로 저장되었습니다.
12번째 이미지가 성공적으로 저장되었습니다.
13번째 이미지가 성공적으로 저장되었습니다.
14번째 이미지가 성공적으로 저장되었습니다.
15번째 이미지가 성공적으로 저장되었습니다.
16번째 이미지가 성공적으로 저장되었습니다.
17번째 이미지가 성공적으로 저장되었습니다.
18번째 이미지가 성공적으로 저장되었습니다.
19번째 이미지가 성공적으로 저장되었습니다.
20번째 이미지가 성공적으로 저장되었습니다.
21번째 이미지가 성공적으로 저장되었습니다.
22번째 이미지가 성공적으로 저장되었습니다.
23번째 이미지가 성공적으로 저장되었습니다.
24번째 이미지가 성공적으로 저장되었습니다.
25번째 이미지가 성공적으로 저장되었습니다.
26번째 이미지가 성공적으로 저장되었습니다.
27번째 이미지가 성공적으로 저장되었습니다.
28번째 이미지가 성공적으로 저장되었습니다.
29번째 이미지가 성공적으로 저장되었습니다.
30번째 이미지가 성공적으로 저장되었습니다.
31번째 이미지가 성공적으로 저장되었습니다.
32번째 이미지가 성공적으로 저장되었습니다.
33번째 이미지가 성공적으로 저장되었습니다.
34번째 이미지가 성공적으로 저장되었습니다.
35번째 이미지가 성공적으로 저장되었습니다.
36번째 이미지가 성공적으로 저장되었습니다.
37번째 이미지가 성공적으로 저장되었습니다.
38번째 이미지가 성공적으로 저장되었습니다.
39번째 이미지가 성공적으로 저장되었습니다.
40번째 이미지가 

In [ ]:
# 이미지 목록을 불러오기
path = "/content/drive/MyDrive/데청캠/Train/페미닌/"
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.jpg')]

for i in range(len(file_list_py)):

  # 입력 이미지 로드
  image = Image.open(path + file_list_py[i])
  # 이미지를 Torch Tensor로 변환
  transform = T.Compose([T.ToTensor()])
  input_image = transform(image)
  # 이미지 배치(batch)로 변환
  input_image = input_image.unsqueeze(0)

  # 모델 예측 수행
  with torch.no_grad():
    prediction = model(input_image)

  boxes = prediction[0]['boxes'] # bounding box 좌표
  scores = prediction[0]['scores'] #객체의 확률 값들

  # 확률 값이 가장 큰 bounding box 좌표 출력
  bounding_box = boxes[find_max_index(scores)]

  # 텐서에서 바운딩 박스 좌표 얻기
  x_min, y_min, x_max, y_max = bounding_box

  # 텐서를 정수로 반환(인덱싱 하려고)
  x_min = math.floor(x_min.item())
  y_min = math.floor(y_min.item())
  x_max = math.floor(x_max.item())
  y_max = math.floor(y_max.item())

  # 이미지 읽기
  image = cv2.imread(path + file_list_py[i])

  # 바운딩 박스 좌표를 이용하여 이미지 crop
  cropped_image = image[y_min:y_max, x_min:x_max]

  # crop된 이미지 저장 경로
  output_path = '/content/drive/MyDrive/데청캠_공유/K-Fashion/페미닌/' + 'cropped_' + file_list_py[i]

  # crop된 이미지 저장
  cv2.imwrite(output_path, cropped_image)

  print(f'{i}번째 이미지가 성공적으로 저장되었습니다.')


0번째 이미지가 성공적으로 저장되었습니다.
1번째 이미지가 성공적으로 저장되었습니다.
2번째 이미지가 성공적으로 저장되었습니다.
3번째 이미지가 성공적으로 저장되었습니다.
4번째 이미지가 성공적으로 저장되었습니다.
5번째 이미지가 성공적으로 저장되었습니다.
6번째 이미지가 성공적으로 저장되었습니다.
7번째 이미지가 성공적으로 저장되었습니다.
8번째 이미지가 성공적으로 저장되었습니다.
9번째 이미지가 성공적으로 저장되었습니다.
10번째 이미지가 성공적으로 저장되었습니다.
11번째 이미지가 성공적으로 저장되었습니다.
12번째 이미지가 성공적으로 저장되었습니다.
13번째 이미지가 성공적으로 저장되었습니다.
14번째 이미지가 성공적으로 저장되었습니다.
15번째 이미지가 성공적으로 저장되었습니다.
16번째 이미지가 성공적으로 저장되었습니다.
17번째 이미지가 성공적으로 저장되었습니다.
18번째 이미지가 성공적으로 저장되었습니다.
19번째 이미지가 성공적으로 저장되었습니다.
20번째 이미지가 성공적으로 저장되었습니다.
21번째 이미지가 성공적으로 저장되었습니다.
22번째 이미지가 성공적으로 저장되었습니다.
23번째 이미지가 성공적으로 저장되었습니다.
24번째 이미지가 성공적으로 저장되었습니다.
25번째 이미지가 성공적으로 저장되었습니다.
26번째 이미지가 성공적으로 저장되었습니다.
27번째 이미지가 성공적으로 저장되었습니다.
28번째 이미지가 성공적으로 저장되었습니다.
29번째 이미지가 성공적으로 저장되었습니다.
30번째 이미지가 성공적으로 저장되었습니다.
31번째 이미지가 성공적으로 저장되었습니다.
32번째 이미지가 성공적으로 저장되었습니다.
33번째 이미지가 성공적으로 저장되었습니다.
34번째 이미지가 성공적으로 저장되었습니다.
35번째 이미지가 성공적으로 저장되었습니다.
36번째 이미지가 성공적으로 저장되었습니다.
37번째 이미지가 성공적으로 저장되었습니다.
38번째 이미지가 성공적으로 저장되었습니다.
39번째 이미지가 성공적으로 저장되었습니다.
40번째 이미지가 

In [ ]:
# 이미지 목록을 불러오기
path = "/content/drive/MyDrive/데청캠/Train/프레피/"
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.jpg')]

for i in range(len(file_list_py)):

  # 입력 이미지 로드
  image = Image.open(path + file_list_py[i])
  # 이미지를 Torch Tensor로 변환
  transform = T.Compose([T.ToTensor()])
  input_image = transform(image)
  # 이미지 배치(batch)로 변환
  input_image = input_image.unsqueeze(0)

  # 모델 예측 수행
  with torch.no_grad():
    prediction = model(input_image)

  boxes = prediction[0]['boxes'] # bounding box 좌표
  scores = prediction[0]['scores'] #객체의 확률 값들

  # 확률 값이 가장 큰 bounding box 좌표 출력
  bounding_box = boxes[find_max_index(scores)]

  # 텐서에서 바운딩 박스 좌표 얻기
  x_min, y_min, x_max, y_max = bounding_box

  # 텐서를 정수로 반환(인덱싱 하려고)
  x_min = math.floor(x_min.item())
  y_min = math.floor(y_min.item())
  x_max = math.floor(x_max.item())
  y_max = math.floor(y_max.item())

  # 이미지 읽기
  image = cv2.imread(path + file_list_py[i])

  # 바운딩 박스 좌표를 이용하여 이미지 crop
  cropped_image = image[y_min:y_max, x_min:x_max]

  # crop된 이미지 저장 경로
  output_path = '/content/drive/MyDrive/데청캠_공유/K-Fashion/프레피/' + 'cropped_' + file_list_py[i]

  # crop된 이미지 저장
  cv2.imwrite(output_path, cropped_image)

  print(f'{i}번째 이미지가 성공적으로 저장되었습니다.')


0번째 이미지가 성공적으로 저장되었습니다.
1번째 이미지가 성공적으로 저장되었습니다.
2번째 이미지가 성공적으로 저장되었습니다.
3번째 이미지가 성공적으로 저장되었습니다.
4번째 이미지가 성공적으로 저장되었습니다.
5번째 이미지가 성공적으로 저장되었습니다.
6번째 이미지가 성공적으로 저장되었습니다.
7번째 이미지가 성공적으로 저장되었습니다.
8번째 이미지가 성공적으로 저장되었습니다.
9번째 이미지가 성공적으로 저장되었습니다.
10번째 이미지가 성공적으로 저장되었습니다.
11번째 이미지가 성공적으로 저장되었습니다.
12번째 이미지가 성공적으로 저장되었습니다.
13번째 이미지가 성공적으로 저장되었습니다.
14번째 이미지가 성공적으로 저장되었습니다.
15번째 이미지가 성공적으로 저장되었습니다.
16번째 이미지가 성공적으로 저장되었습니다.
17번째 이미지가 성공적으로 저장되었습니다.
18번째 이미지가 성공적으로 저장되었습니다.
19번째 이미지가 성공적으로 저장되었습니다.
20번째 이미지가 성공적으로 저장되었습니다.
21번째 이미지가 성공적으로 저장되었습니다.
22번째 이미지가 성공적으로 저장되었습니다.
23번째 이미지가 성공적으로 저장되었습니다.
24번째 이미지가 성공적으로 저장되었습니다.
25번째 이미지가 성공적으로 저장되었습니다.
26번째 이미지가 성공적으로 저장되었습니다.
27번째 이미지가 성공적으로 저장되었습니다.
28번째 이미지가 성공적으로 저장되었습니다.
29번째 이미지가 성공적으로 저장되었습니다.
30번째 이미지가 성공적으로 저장되었습니다.
31번째 이미지가 성공적으로 저장되었습니다.
32번째 이미지가 성공적으로 저장되었습니다.
33번째 이미지가 성공적으로 저장되었습니다.
34번째 이미지가 성공적으로 저장되었습니다.
35번째 이미지가 성공적으로 저장되었습니다.
36번째 이미지가 성공적으로 저장되었습니다.
37번째 이미지가 성공적으로 저장되었습니다.
38번째 이미지가 성공적으로 저장되었습니다.
39번째 이미지가 성공적으로 저장되었습니다.
40번째 이미지가 

In [ ]:
# 이미지 목록을 불러오기
path = "/content/drive/MyDrive/데청캠/Train/히피/"
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.jpg')]

for i in range(len(file_list_py)):

  # 입력 이미지 로드
  image = Image.open(path + file_list_py[i])
  # 이미지를 Torch Tensor로 변환
  transform = T.Compose([T.ToTensor()])
  input_image = transform(image)
  # 이미지 배치(batch)로 변환
  input_image = input_image.unsqueeze(0)

  # 모델 예측 수행
  with torch.no_grad():
    prediction = model(input_image)

  boxes = prediction[0]['boxes'] # bounding box 좌표
  scores = prediction[0]['scores'] #객체의 확률 값들

  # 확률 값이 가장 큰 bounding box 좌표 출력
  bounding_box = boxes[find_max_index(scores)]

  # 텐서에서 바운딩 박스 좌표 얻기
  x_min, y_min, x_max, y_max = bounding_box

  # 텐서를 정수로 반환(인덱싱 하려고)
  x_min = math.floor(x_min.item())
  y_min = math.floor(y_min.item())
  x_max = math.floor(x_max.item())
  y_max = math.floor(y_max.item())

  # 이미지 읽기
  image = cv2.imread(path + file_list_py[i])

  # 바운딩 박스 좌표를 이용하여 이미지 crop
  cropped_image = image[y_min:y_max, x_min:x_max]

  # crop된 이미지 저장 경로
  output_path = '/content/drive/MyDrive/데청캠_공유/K-Fashion/히피/' + 'cropped_' + file_list_py[i]

  # crop된 이미지 저장
  cv2.imwrite(output_path, cropped_image)

  print(f'{i}번째 이미지가 성공적으로 저장되었습니다.')


0번째 이미지가 성공적으로 저장되었습니다.
1번째 이미지가 성공적으로 저장되었습니다.
2번째 이미지가 성공적으로 저장되었습니다.
3번째 이미지가 성공적으로 저장되었습니다.
4번째 이미지가 성공적으로 저장되었습니다.
5번째 이미지가 성공적으로 저장되었습니다.
6번째 이미지가 성공적으로 저장되었습니다.
7번째 이미지가 성공적으로 저장되었습니다.
8번째 이미지가 성공적으로 저장되었습니다.
9번째 이미지가 성공적으로 저장되었습니다.
10번째 이미지가 성공적으로 저장되었습니다.
11번째 이미지가 성공적으로 저장되었습니다.
12번째 이미지가 성공적으로 저장되었습니다.
13번째 이미지가 성공적으로 저장되었습니다.
14번째 이미지가 성공적으로 저장되었습니다.
15번째 이미지가 성공적으로 저장되었습니다.
16번째 이미지가 성공적으로 저장되었습니다.
17번째 이미지가 성공적으로 저장되었습니다.
18번째 이미지가 성공적으로 저장되었습니다.
19번째 이미지가 성공적으로 저장되었습니다.
20번째 이미지가 성공적으로 저장되었습니다.
21번째 이미지가 성공적으로 저장되었습니다.
22번째 이미지가 성공적으로 저장되었습니다.
23번째 이미지가 성공적으로 저장되었습니다.
24번째 이미지가 성공적으로 저장되었습니다.
25번째 이미지가 성공적으로 저장되었습니다.
26번째 이미지가 성공적으로 저장되었습니다.
27번째 이미지가 성공적으로 저장되었습니다.
28번째 이미지가 성공적으로 저장되었습니다.
29번째 이미지가 성공적으로 저장되었습니다.
30번째 이미지가 성공적으로 저장되었습니다.
31번째 이미지가 성공적으로 저장되었습니다.
32번째 이미지가 성공적으로 저장되었습니다.
33번째 이미지가 성공적으로 저장되었습니다.
34번째 이미지가 성공적으로 저장되었습니다.
35번째 이미지가 성공적으로 저장되었습니다.
36번째 이미지가 성공적으로 저장되었습니다.
37번째 이미지가 성공적으로 저장되었습니다.
38번째 이미지가 성공적으로 저장되었습니다.
39번째 이미지가 성공적으로 저장되었습니다.
40번째 이미지가 

In [ ]:
# 이미지 목록을 불러오기
path = "/content/drive/MyDrive/데청캠/Train/힙합/"
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.jpg')]

for i in range(len(file_list_py)):

  # 입력 이미지 로드
  image = Image.open(path + file_list_py[i])
  # 이미지를 Torch Tensor로 변환
  transform = T.Compose([T.ToTensor()])
  input_image = transform(image)
  # 이미지 배치(batch)로 변환
  input_image = input_image.unsqueeze(0)

  # 모델 예측 수행
  with torch.no_grad():
    prediction = model(input_image)

  boxes = prediction[0]['boxes'] # bounding box 좌표
  scores = prediction[0]['scores'] #객체의 확률 값들

  # 확률 값이 가장 큰 bounding box 좌표 출력
  bounding_box = boxes[find_max_index(scores)]

  # 텐서에서 바운딩 박스 좌표 얻기
  x_min, y_min, x_max, y_max = bounding_box

  # 텐서를 정수로 반환(인덱싱 하려고)
  x_min = math.floor(x_min.item())
  y_min = math.floor(y_min.item())
  x_max = math.floor(x_max.item())
  y_max = math.floor(y_max.item())

  # 이미지 읽기
  image = cv2.imread(path + file_list_py[i])

  # 바운딩 박스 좌표를 이용하여 이미지 crop
  cropped_image = image[y_min:y_max, x_min:x_max]

  # crop된 이미지 저장 경로
  output_path = '/content/drive/MyDrive/데청캠_공유/K-Fashion/힙합/' + 'cropped_' + file_list_py[i]

  # crop된 이미지 저장
  cv2.imwrite(output_path, cropped_image)

  print(f'{i}번째 이미지가 성공적으로 저장되었습니다.')


0번째 이미지가 성공적으로 저장되었습니다.
1번째 이미지가 성공적으로 저장되었습니다.
2번째 이미지가 성공적으로 저장되었습니다.
3번째 이미지가 성공적으로 저장되었습니다.
4번째 이미지가 성공적으로 저장되었습니다.
5번째 이미지가 성공적으로 저장되었습니다.
6번째 이미지가 성공적으로 저장되었습니다.
7번째 이미지가 성공적으로 저장되었습니다.
8번째 이미지가 성공적으로 저장되었습니다.
9번째 이미지가 성공적으로 저장되었습니다.
10번째 이미지가 성공적으로 저장되었습니다.
11번째 이미지가 성공적으로 저장되었습니다.
12번째 이미지가 성공적으로 저장되었습니다.
13번째 이미지가 성공적으로 저장되었습니다.
14번째 이미지가 성공적으로 저장되었습니다.
15번째 이미지가 성공적으로 저장되었습니다.
16번째 이미지가 성공적으로 저장되었습니다.
17번째 이미지가 성공적으로 저장되었습니다.
18번째 이미지가 성공적으로 저장되었습니다.
19번째 이미지가 성공적으로 저장되었습니다.
20번째 이미지가 성공적으로 저장되었습니다.
21번째 이미지가 성공적으로 저장되었습니다.
22번째 이미지가 성공적으로 저장되었습니다.
23번째 이미지가 성공적으로 저장되었습니다.
24번째 이미지가 성공적으로 저장되었습니다.
25번째 이미지가 성공적으로 저장되었습니다.
26번째 이미지가 성공적으로 저장되었습니다.
27번째 이미지가 성공적으로 저장되었습니다.
28번째 이미지가 성공적으로 저장되었습니다.
29번째 이미지가 성공적으로 저장되었습니다.
30번째 이미지가 성공적으로 저장되었습니다.
31번째 이미지가 성공적으로 저장되었습니다.
32번째 이미지가 성공적으로 저장되었습니다.
33번째 이미지가 성공적으로 저장되었습니다.
34번째 이미지가 성공적으로 저장되었습니다.
35번째 이미지가 성공적으로 저장되었습니다.
36번째 이미지가 성공적으로 저장되었습니다.
37번째 이미지가 성공적으로 저장되었습니다.
38번째 이미지가 성공적으로 저장되었습니다.
39번째 이미지가 성공적으로 저장되었습니다.
40번째 이미지가 